In [ ]:
from utils import extract_all_features_for_count_threshold, extract_features_for_dist_threshold, get_custom_matrix_features
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
# read in data
from utils import get_matrix_features
path_to_data = os.path.abspath('data')
path_to_colonies = os.path.abspath('data/colonies')

In [ ]:

import pandas as pd
feature_list = ['']
threshold_mode = 'count'
num_nn = 4
# colony 0
colony0_segmentation_path = '/home/farzaneh/Documents/Bread/bread/src/bread/tests/data/V2022_09_19_HTB2_mCh_MYO1-GFP_50_ms/FOV0_segmentation_T0_to_T146_trimmed.h5'
colony0_lineage_GT_path = '/home/farzaneh/Documents/Bread/bread/src/bread/tests/data/V2022_09_19_HTB2_mCh_MYO1-GFP_50_ms/FOV0_lineage_T0_to_T146.csv'
colony0_lineage_gt = pd.read_csv(colony0_lineage_GT_path).rename(
    columns={'parent_id': 'parent_GT'})
colony0_lineage_gt['colony'] = [0 for i in range(len(colony0_lineage_gt))]

# colony0_features, f_list = extract_all_features_for_count_threshold(
#     colony0_segmentation_path, num_nn=num_nn)
# colony0_features['colony'] = [0 for i in range(len(colony0_features))]

# colony0_matrix_features = get_custom_matrix_features(
#     colony0_features, colony0_lineage_gt, f_list)

# colony 1 to 5
# load the 5 colony ground truth
colonies_gt = pd.DataFrame()
for i in [1, 2, 3, 4, 5]:
    temp_colony = pd.read_csv(os.path.join(
        path_to_colonies, 'colony00{}_lineage.csv'.format(i)))
    temp_colony['colony'] = i
    colonies_gt = pd.concat([colonies_gt, temp_colony])
colonies_gt = colonies_gt.reset_index(drop=True)
colonies_gt.rename(columns={'# parent_id': 'parent_GT'}, inplace=True)


# extract features for each colony and save them to a csv file. This takes a while. But after this, we only need to load the csv files. every time.
    # save to csv
    # colony_candidate_features[i-1].to_csv(os.path.join(
    #     path_to_colonies, 'colony00{}_candidate_features_19_{}_{}_nn_{}_threshold.csv'.format(i, threshold_mode, num_nn, 10)), index=False)
    # colony_candidate_features[i-1] = pd.read_csv(os.path.join(path_to_colonies, 'colony00{}_candidate_features_{}_{}_nn.csv'.format(i, threshold_mode, num_nn)))



In [ ]:

_19_f_list = ['dist_0','dist_max','dist_min','dist_std','poly_fit_budcm_candidcm','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']
# merge colonies features and colony0 features and save in one csv file
all_candidate_features = pd.read_csv(os.path.join(path_to_data, 'all_candidate_features_19_{}_{}_nn_{}_thresh.csv'.format(threshold_mode, num_nn, 10)))
all_gt = pd.concat([colony0_lineage_gt,colonies_gt]).reset_index()
# all_candidate_features.to_csv(os.path.join(path_to_data, 'all_candidate_features_19_{}_{}_nn_{}_thresh.csv'.format(threshold_mode, num_nn, 10)), index=False)
all_matrix_features = get_custom_matrix_features(all_candidate_features, all_gt, _19_f_list)
all_matrix_features

## external data


In [ ]:
f_list = ['dist_0','dist_std','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']
external_subset_lineage_GT_path_edited = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/lineage/lineage_budlum_edited_by_code.csv'
external_subset_features_path = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/features/features_19.csv'
external_subset_segmentation_path = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/segmentation2/stacked/mask_stack _edited.h5'
threshold_mode = 'count'
num_nn = 4
path_to_features = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/lineage/'

external_subset_features ,_ =  extract_all_features_for_count_threshold(
    external_subset_segmentation_path, num_nn=num_nn)
external_subset_features['colony'] = 10
external_subset_lineage_gt = pd.read_csv(external_subset_lineage_GT_path_edited)
external_subset_lineage_gt['colony'] = 10
path_to_features = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/lineage/'

external_subset_features.to_csv(os.path.join(
    path_to_features, 'external_subset_candidate_features_{}_{}_nn_{}_features.csv'.format(threshold_mode, num_nn, 19)))
# create matrix features for the external subset
external_subset_matrix_features = get_custom_matrix_features(
    external_subset_features, external_subset_lineage_gt, f_list)

# Train XGboost with new model


In [ ]:
from model_xgboost import plot_feature_importance, run_xgboost, cv_xgboost

In [ ]:
# train and test for all colonies
f_list = _19_f_list
all_matrix_features = get_custom_matrix_features(all_candidate_features, all_gt, f_list)
mean_score, std_score, models_all = cv_xgboost(all_matrix_features['features'].to_numpy(),
                                               all_matrix_features['parent_index_in_candidates'].to_numpy())

In [ ]:
for model in models_all:
    plot_feature_importance(model, figsize=(15,5) ,title='custom model')


In [ ]:
eliminate_result={}
param = {'objective': 'multi:softmax', 'num_class': 4, 'max_depth': 5, 'min_child_weight': 2, 'eval_metric': 'merror'}
for each in _19_f_list:
    f_list = list(set(_19_f_list)-set([each]))
    custom_matrix = get_custom_matrix_features(all_candidate_features, all_gt, f_list)
    mean_score, std_score, models_all = cv_xgboost(custom_matrix['features'].to_numpy(),
                                               custom_matrix['parent_index_in_candidates'].to_numpy())
    eliminate_result[each] = mean_score


In [ ]:

# Sort the keys based on the values in descending order
sorted_keys = sorted(eliminate_result, key=eliminate_result.get, reverse=True)

# Create a new dictionary with the sorted keys and values
sorted_dict = {k: eliminate_result[k] for k in sorted_keys}

sorted_dict

In [ ]:
# train for only first 10 features
# train and test for all colonies
f_list = ['position_bud_std','poly_fit_budcm_budpt','poly_fit_expansion_vector', 'orientation_bud_last_minus_first','position_bud_first', 'dist_std', 'poly_fit_budcm_candidcm', 'position_bud_max', 'orientation_bud_max', 'orientation_bud_min']
all_matrix_features = get_custom_matrix_features(all_candidate_features, all_gt, f_list)
mean_score, std_score, models_all = cv_xgboost(all_matrix_features['features'].to_numpy(),
                                               all_matrix_features['parent_index_in_candidates'].to_numpy())

In [ ]:
# train for only first 18 features
# train and test for all colonies
f_list = ['dist_max','dist_0','orientation_bud_first','dist_min','position_bud_std','orientation_bud_std','position_bud_min','position_bud_last','orientation_bud_last','poly_fit_budcm_budpt','poly_fit_expansion_vector', 'orientation_bud_last_minus_first','position_bud_first', 'dist_std', 'poly_fit_budcm_candidcm', 'position_bud_max', 'orientation_bud_max', 'orientation_bud_min']
all_matrix_features = get_custom_matrix_features(all_candidate_features, all_gt, f_list)
mean_score, std_score, models_all = cv_xgboost(all_matrix_features['features'].to_numpy(),
                                               all_matrix_features['parent_index_in_candidates'].to_numpy())

In [ ]:
# train for only meaningfull features
# train and test for all colonies
f_list = ['dist_max','dist_0','orientation_bud_first','position_bud_std','orientation_bud_std','poly_fit_expansion_vector', 'orientation_bud_last_minus_first', 'dist_std', 'poly_fit_budcm_candidcm',]
all_matrix_features = get_custom_matrix_features(all_candidate_features, all_gt, f_list)
mean_score, std_score, models_all = cv_xgboost(all_matrix_features['features'].to_numpy(),
                                               all_matrix_features['parent_index_in_candidates'].to_numpy())